In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
def pairwise_dist(x, y):  # [5 pts]
    """
        Args:
            x: N x D numpy array
            y: M x D numpy array
        Return:
                dist: N x M array, where dist2[i, j] is the euclidean distance between
                x[i, :] and y[j, :]
    """
    #raise NotImplementedError
    return np.linalg.norm(x[:, None, :] - y[None, :, :], axis=-1)

In [5]:
cities = pd.read_csv("data/ds2.csv")
cities = cities.rename(columns = {'Unnamed: 0':'city_id'})
cities.shape

(950, 50)

In [6]:
data_us = pd.read_csv('data/uscities.csv')

data_us = data_us[['city', 'state_id', 'state_name', 'county_name', 'lat', 'lng']]
data_us['city_name_id'] = data_us.apply(lambda x: x['city'] + ", " + x['state_name'] + ", " + x['county_name'] + " County", axis = 1)
data_us['city_state_id'] = data_us.apply(lambda x: x['city'] + ", " + x['state_name'], axis = 1)

data_cities = cities.copy()
data_cities['city_name_id'] = data_cities.apply(lambda x: x['City'] + ", " + x['State'] + ", " + x['County'], axis = 1)
data_cities['city_state_id'] = data_cities.apply(lambda x: x['City'] + ", " + x['State'], axis = 1)

city_all = data_cities.merge(data_us, on = 'city_name_id', how = 'left')
city_all = city_all.rename(columns = {'city_state_id_x':'city_state'})
city_all = city_all[~city_all['lat'].isna()]
city_all = city_all[~city_all['lng'].isna()]
city_all = city_all.drop(columns = ['city_id', 'city_state_id_y'])
city_all = city_all.reset_index(drop = True)
city_all = city_all.reset_index()
city_all = city_all.rename(columns = {'index':'city_id'})
city_all.to_csv('data/city_all.csv', index = False)
city_all.shape

(919, 58)

In [7]:
city_all[city_all['lng'].isna()][['city_id', 'city_state', 'lat', 'lng']]

,city_id,city_state,lat,lng


In [8]:
meta_columns = ['city_id', 'State', 'County', 'State Abbreviation', 'city_name_id', 'city_state', 'city', 'City', 'state_id', 'county_name', 'state_name', 'lat', 'lng']
city_metadata = city_all[meta_columns]
city_metadata.to_csv('data/city_metadata.csv', index = False)


In [9]:
def id_to_city(city_id):
    return pd.read_csv('data/city_metadata.csv').iloc[[city_id]]
id_to_city(10)

,city_id,State,County,State Abbreviation,city_name_id,city_state,city,City,state_id,county_name,state_name,lat,lng
10,10,Texas,Travis County,TX,"Austin, Texas, Travis County","Austin, Texas",Austin,Austin,TX,Travis,Texas,30.3005,-97.7522


In [10]:
city_df = city_all.drop(columns = meta_columns)
# delete_cols = ['city_state_id_y', 'state_id', 'city_name_id', 'city', 'state_name', 'county_name']
# city_df = city_df.drop(columns = delete_cols)
city_df.head()

,Average Household Income 2020/2021,"Percent Change in Household Income, 1984-2019","Average Yearly Percent Change in Household Income, 1984-2019",MinWage,Average House Price 2020/2021,"Percent Change in Housing Prices, 2000-2019","Average Yearly Percent Change in Housing Prices, 2000-2019",Violent Crime Total,Property Crime Total,Property Crime Per Capita,...,Walk,OtherTransp,WorkAtHome,MeanCommute,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,Population
0,-0.136317,0.851226,0.851226,0.855108,1.045961,1.230131,1.230131,18.378204,14.963376,-0.662457,...,1.627811,-0.553042,-1.312925,3.828220,-0.203596,0.226288,0.006249,-0.591108,-0.016201,23.673822
1,0.737962,0.014144,0.014144,1.139948,1.393001,1.882940,1.882940,10.946839,9.795925,-0.165898,...,0.092612,0.256218,0.250712,1.128201,-0.239062,-0.512411,1.786000,0.071967,0.417355,10.737820
2,0.457552,0.224546,0.224546,0.427849,-0.363198,-0.868798,-0.868798,NaN,NaN,NaN,...,0.845506,0.347146,-0.302575,1.687189,0.671963,-0.320016,-0.911262,-0.591108,1.564181,7.403277
3,-0.416098,-0.755247,-0.755247,-1.263387,-0.588687,-0.388227,-0.388227,10.953656,11.118278,1.514108,...,-0.560289,-0.034752,-0.771666,0.529134,0.669746,-0.873447,0.359989,-0.093802,-0.132749,5.907363
4,-0.042637,1.850164,1.850164,-1.263387,-0.602243,0.856306,0.856306,NaN,NaN,NaN,...,3.374761,1.256428,-0.807750,1.533204,0.654230,-0.168001,-1.204202,-1.088414,2.240155,4.068471


In [11]:
city_columns = list(enumerate(city_df.columns))
city_columns

[(0, 'Average Household Income 2020/2021'),
 (1, 'Percent Change in Household Income, 1984-2019'),
 (2, 'Average Yearly Percent Change in Household Income, 1984-2019'),
 (3, 'MinWage'),
 (4, 'Average House Price 2020/2021'),
 (5, 'Percent Change in Housing Prices, 2000-2019'),
 (6, 'Average Yearly Percent Change in Housing Prices, 2000-2019'),
 (7, 'Violent Crime Total'),
 (8, 'Property Crime Total'),
 (9, 'Property Crime Per Capita'),
 (10, 'Violent Crime Per Capita'),
 (11, 'Total Crime Per Capita'),
 (12, 'TotalPop'),
 (13, 'Men'),
 (14, 'Women'),
 (15, 'VotingAgeCitizen'),
 (16, 'Employed'),
 (17, 'Hispanic'),
 (18, 'White'),
 (19, 'Black'),
 (20, 'Native'),
 (21, 'Asian'),
 (22, 'Pacific'),
 (23, 'Income'),
 (24, 'IncomePerCap'),
 (25, 'Poverty'),
 (26, 'ChildPoverty'),
 (27, 'Professional'),
 (28, 'Service'),
 (29, 'Office'),
 (30, 'Construction'),
 (31, 'Production'),
 (32, 'Drive'),
 (33, 'Carpool'),
 (34, 'Transit'),
 (35, 'Walk'),
 (36, 'OtherTransp'),
 (37, 'WorkAtHome'),
 (

In [12]:
city_np = np.array(city_df)
city_np.shape

(919, 45)

In [13]:
city_np

array([[-1.36317125e-01,  8.51225950e-01,  8.51225950e-01, ...,
        -5.91107781e-01, -1.62011840e-02,  2.36738216e+01],
       [ 7.37961649e-01,  1.41441490e-02,  1.41441490e-02, ...,
         7.19670230e-02,  4.17354867e-01,  1.07378196e+01],
       [ 4.57551781e-01,  2.24546057e-01,  2.24546057e-01, ...,
        -5.91107781e-01,  1.56418055e+00,  7.40327713e+00],
       ...,
       [ 1.35302461e+00,  2.79941865e+00,  2.79941865e+00, ...,
         2.37735724e-01, -1.24694094e+00, -2.69370536e-01],
       [-3.43854299e-01,  7.78758422e-01,  7.78758422e-01, ...,
        -7.56876483e-01,  4.90653190e-02, -2.69565082e-01],
       [-1.32623241e+00, -6.54708389e-01, -6.54708389e-01, ...,
         1.23234793e+00, -2.44633941e-01, -2.69596553e-01]])

In [14]:
cities_dist = None
cities_nn = None
if True:
    cities_dist = pairwise_dist(city_np, city_np) #.tofile('data/pairwise_city_dist.csv', sep = ',')
    #np.savetxt("data/city_pairwise_dist.csv", cities_dist, delimiter=",")
    temp = pd.DataFrame(cities_dist)
    temp.to_csv("data/city_distances.csv", index = False)
    cities_nn = pd.DataFrame(np.argsort(cities_dist))
    #np.savetxt("data/city_nn_ranked.csv", cities_nn, delimiter=",")
    cities_nn.to_csv("data/city_nn.csv", index = False)
    cities_dist = temp
    del temp
else:
    cities_dist = np.array(pd.read_csv("data/city_distances.csv"))
    cities_nn = np.array(pd.read_csv("data/city_nn.csv"))

In [15]:
np.sum(cities_nn, axis = 1)

0      421821
1      421821
2      421821
3      421821
4      421821
        ...  
914    421821
915    421821
916    421821
917    421821
918    421821
Length: 919, dtype: int64

In [16]:
def get_nn(city_id, n = 5):
    """
        Args:
            city_id: int
            n: int
        Return:
            nearest_neighbors: (n,) ndarray, where nearest_neighbors[i] is the city_id of the i'th most similar city
    """

    #Don't include the first column since it will always be itself (distances between a city and itself is 0 miles)
    dta = cities_nn.iloc[city_id, 1:n+1]

    return dta
    
get_nn(25, 5)

1    605
2    606
3    607
4    608
5    609
Name: 25, dtype: int64

In [17]:
cities_nn.head()

,0,1,2,3,4,5,6,7,8,9,...,909,910,911,912,913,914,915,916,917,918
0,0,1,3,5,6,8,7,13,20,21,...,719,721,511,230,727,507,733,735,708,460
1,1,3,5,6,8,7,33,20,16,10,...,648,215,221,632,630,223,625,624,646,212
2,0,605,606,607,608,609,610,611,612,613,...,305,306,307,308,309,310,311,312,314,918
3,3,6,8,1,5,19,16,21,17,12,...,146,507,147,70,504,771,502,773,498,221
4,0,605,606,607,608,609,610,611,612,613,...,305,306,307,308,309,310,311,312,314,918


In [18]:
city_metadata.columns

Index(['city_id', 'State', 'County', 'State Abbreviation', 'city_name_id',
       'city_state', 'city', 'City', 'state_id', 'county_name', 'state_name',
       'lat', 'lng'],
      dtype='object')

In [19]:
nearest_neighbors = pd.concat([city_metadata, cities_nn.iloc[:, 0:21]], axis = 1)

nearest_neighbors['city_county'] = nearest_neighbors.apply(lambda x: x['City'] + ', ' + x['County'], axis = 1)
nearest_neighbors['city_st_abbr'] = nearest_neighbors.apply(lambda x: x['City'] + ', ' + x['State Abbreviation'], axis = 1)

nearest_neighbors.to_csv('data/nearest_neighbors.csv', index = False)

In [21]:
pd.read_csv('data/nearest_neighbors.csv').columns

Index(['city_id', 'State', 'County', 'State Abbreviation', 'city_name_id',
       'city_state', 'city', 'City', 'state_id', 'county_name', 'state_name',
       'lat', 'lng', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       'city_county', 'city_st_abbr'],
      dtype='object')